In [15]:
import pandas as pd
import glob
import os

m_store = pd.read_csv("d:\\data_analysis_lab\\ML_100\\data\\m_store.csv")
m_area = pd.read_csv("d:\\data_analysis_lab\\ML_100\\data\\m_area.csv")


current_dir = os.getcwd()
# 찾을 파일 패턴 만들기
tbl_order_file = os.path.join(current_dir + "\\data\\", 'tbl_order_*.csv')
# 해당하는 패턴과 일치하는 모든 파일 경로를 리스트로 변환
tbl_order_files = glob.glob(tbl_order_file)
order_all  = pd.DataFrame()
for file in tbl_order_files :
    order_tmp = pd.read_csv(file)
    order_all = pd.concat([order_all, order_tmp], ignore_index=True)
    order_all.head()
    


In [16]:
order_all.head()

,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status
0,34104383,11,C65806632,57,61573513,2021-04-01 11:00,2021-04-01 11:39,1,28270,1
1,70652318,59,C09760173,37,54068709,2021-04-01 11:00,2021-04-01 11:34,0,28270,2
2,71640388,195,C61227084,17,93678366,2021-04-01 11:00,2021-04-01 11:54,0,26470,9
3,75673365,127,C64119972,17,5287952,2021-04-01 11:00,2021-04-01 11:17,0,23080,2
4,9077529,174,C10231192,18,18248867,2021-04-01 11:00,2021-04-01 11:35,0,46920,2


In [20]:
# 유지 보수용 매장 데이터 삭제  
order_all = order_all.loc[order_all['store_id'] != 999]

order_all = pd.merge(order_all, m_store, on="store_id", how='left')
order_all = pd.merge(order_all, m_area, on="area_cd", how='left')

In [21]:
# 마스터에 없는 코드에 대응한 문자열 설정

order_all.loc[order_all['takeout_flag'] == 0, 'takeout_name'] = "delivery"
order_all.loc[order_all['takeout_flag'] == 1, 'takeout_name'] = "takeout"


order_all.loc[order_all['status'] == 0, 'status_name'] = '주문접수'
order_all.loc[order_all['status'] == 1, 'status_name'] = '결제완료'
order_all.loc[order_all['status'] == 2, 'status_name'] = '배달완료'
order_all.loc[order_all['status'] == 3, 'status_name'] = '주문취소'
print(order_all.head())

   order_id  store_id customer_id  coupon_cd  sales_detail_id  \
0  34104383        11   C65806632         57         61573513   
1  70652318        59   C09760173         37         54068709   
2  71640388       195   C61227084         17         93678366   
3  75673365       127   C64119972         17          5287952   
4   9077529       174   C10231192         18         18248867   

  order_accept_date    delivered_date  takeout_flag  total_amount  status  \
0  2021-04-01 11:00  2021-04-01 11:39             1         28270       1   
1  2021-04-01 11:00  2021-04-01 11:34             0         28270       2   
2  2021-04-01 11:00  2021-04-01 11:54             0         26470       9   
3  2021-04-01 11:00  2021-04-01 11:17             0         23080       2   
4  2021-04-01 11:00  2021-04-01 11:35             0         46920       2   

  store_name_x area_cd_x store_name_y area_cd_y store_name area_cd wide_area  \
0         자양로점        SL         자양로점        SL       자양로점      SL

In [22]:
import openpyxl

# 엑셀파일 만들기 
wb = openpyxl.Workbook()
ws = wb['Sheet']
ws.cell(1,1).value = "파일 쓰기 테스트입니다"
wb.save("d:\\data_analysis_lab\\ML_100\\data\\test.xlsx")
wb.close()

In [23]:
wb = openpyxl.load_workbook("d:\\data_analysis_lab\\ML_100\\data\\test.xlsx", read_only=True)
ws = wb['Sheet']
print(ws.cell(1,1).value)
wb.close()

파일 쓰기 테스트입니다


In [25]:
# 테스트 데이터 준비 

store_id =1 
store_df = order_all.loc[order_all['store_id'] == store_id].copy()
store_name = store_df['store_name'].unique()[0]
store_sales_total = store_df.loc[store_df['status'].isin([1,2])]['total_amount'].sum()
store_sales_takeout = store_df.loc[store_df['status'] == 1]['total_amount'].sum()
store_sales_delivery = store_df.loc[store_df['status'] == 2]['total_amount'].sum()
print(f'매출확인 {store_sales_total} =', f'{store_sales_takeout + store_sales_delivery}')

ouput_df = store_df[['order_accept_date', 'customer_id', 'total_amount', 'takeout_name', 'status_name']]
ouput_df.head()

매출확인 90101050 = 90101050


,order_accept_date,customer_id,total_amount,takeout_name,status_name
48,2021-04-01 11:03,C81203759,30500,takeout,결제완료
435,2021-04-01 11:32,C93103282,26150,takeout,결제완료
627,2021-04-01 11:50,C33623566,41440,delivery,NaN
642,2021-04-01 11:51,C64109769,33000,delivery,배달완료
976,2021-04-01 12:21,C46929234,26030,delivery,NaN


In [39]:
from openpyxl.utils.dataframe import dataframe_to_rows

store_title = f'{store_id}_{store_name}'

wb = openpyxl.Workbook()
ws = wb.active
ws.title = store_title

ws.cell(1,1).value = f'{store_title} 매출 데이터'

rows = dataframe_to_rows(ouput_df, index=False, header=True)

row_start = 3
col_start = 2

for row_no, row in enumerate(rows, row_start) :
    for col_no, value in enumerate(row, col_start) : 
        ws.cell(row_no, col_no).value = value

filename = f'd:\\data_analysis_lab\\ML_100\\data\\{store_title}.xlsx'
wb.save(filename)
wb.close()

In [37]:
from openpyxl.styles import PatternFill, Border, Side, Font

openpyxl.load_workbook(filename)
ws = wb[store_title]

side = Side(style='thin', color='008080')
border = Border(top=side, bottom=side, left=side, right=side)

for row in ws :
    for cell in row:
        if ws[cell.coordinate].value :
            ws[cell.coordinate].border =  border

In [40]:
ws.cell(1,1).font = Font(bold=True, color="008080")

cell = ws.cell(3,2)
cell.fill = PatternFill(patternType='solid', fgColor='008080')
cell.value = '주문 접수 일시'
cell.font = Font(bold=True, color='FFFFFF')

cell = ws.cell(3,3)
cell.fill = PatternFill(patternType='solid', fgColor='008080')
cell.value = '고객 ID'
cell.font = Font(bold=True, color='FFFFFF')

cell = ws.cell(3,4)
cell.fill = PatternFill(patternType='solid', fgColor='008080')
cell.value = '수입 총액'
cell.font = Font(bold=True, color='FFFFFF')

cell = ws.cell(3,5)
cell.fill = PatternFill(patternType='solid', fgColor='008080')
cell.value = '주문 타입'
cell.font = Font(bold=True, color='FFFFFF')

cell = ws.cell(3,6)
cell.fill = PatternFill(patternType='solid', fgColor='008080')
cell.value = '주문 상태'
cell.font = Font(bold=True, color='FFFFFF')

ws.column_dimensions['A'].width = 20
ws.column_dimensions['B'].width = 20
ws.column_dimensions['C'].width = 12
ws.column_dimensions['D'].width = 12
ws.column_dimensions['E'].width = 12
ws.column_dimensions['F'].width = 12


print("파일에 저장")
wb.save(filename)
wb.close()

파일에 저장


In [44]:
# 매출 이외의 데이터 출력

def cal_dalta(t):
    t1, t2 = t
    delta = t2 - t1
    return delta.total_seconds() / 60


store_df.loc[:, 'order_accept_datetime'] = pd.to_datetime(store_df['order_accept_date'])
store_df.loc[:, 'delivered_datetime'] = pd.to_datetime(store_df['delivered_date'])
store_df.loc[:, 'delta'] = store_df[['order_accept_datetime', 'delivered_datetime']].apply(cal_dalta, axis=1)

delivery_time = store_df.groupby(['store_id'])['delta'].describe()
delivery_time

,count,mean,std,min,25%,50%,75%,max
store_id,,,,,,,,
1,3553.0,34.46327,14.502363,10.0,22.0,34.0,47.0,59.0


In [50]:
openpyxl.load_workbook(filename)
ws = wb[store_title]

cell = ws.cell(1, 7)
cell.value = f'배달시간'
cell.font = Font(bold=True, color = '008080')

rows = dataframe_to_rows(delivery_time, index=False, header=True)

In [51]:
# 표 삽입 위치 

row_start = 3
col_start = 8

for row_no, row in enumerate(rows, row_start) :
    for col_no, value in enumerate(row, col_start) :
        cell = ws.cell(row_no, col_no)
        cell.value = value
        cell.border = border
        if row_no == row_start :
            cell.fill = PatternFill(patternType='solid', fgColor='008080')
            cell.font = Font(bold=True, color='FFFFFF')
            
filename = f'd:\\data_analysis_lab\\ML_100\\data\\{store_title}.xlsx'
wb.save(filename)
wb.close()

In [55]:
# 문제가 있는 위치를 빨간색으로 출력 

openpyxl.load_workbook(filename)
ws = wb[store_title]

rows = dataframe_to_rows(ouput_df, index=False, header=True)

row_start = 3
col_start = 2

for row_no, row in enumerate(rows, row_start) :
    if row_no == row_start : 
        continue
    for col_no, value in enumerate(row, col_start) :
        ws.cell(row_no, col_no).value = value
        if value == '결제완료':
            ws.cell(row_no, col_no).font = Font(bold=True, color='FF0000')

filename = f'd:\\data_analysis_lab\\ML_100\\data\\{store_title}.xlsx'
wb.save(filename)
wb.close()